In [ ]:
#%%1. Importing Libraries
#In this Notebook we will use a Pre Trained Model like Resnet50 for Classification of Dog Breed.

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, InputLayer
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

import warnings
warnings.filterwarnings("ignore")

In [ ]:
plt.rcParams['font.size'] = 10

In [ ]:
# %% 2. Fetching Data

In [ ]:
fpath = r"C:\Users\pc\Downloads\archive (1)\Images"
random_seed = 42

In [ ]:
img_size = 224
batch_size =64
train = tf.keras.utils.image_dataset_from_directory(
  fpath,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_size, img_size),
  batch_size=batch_size,
  label_mode ="categorical"
)

In [ ]:
val = tf.keras.utils.image_dataset_from_directory(
  fpath,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_size, img_size),
  batch_size=batch_size,
  label_mode ="categorical"
)

In [ ]:
# %% 3. Plotting Data

In [ ]:
class_names = train.class_names

In [ ]:
names=[]
for name in class_names :
    # Check if the name contains the delimiter before splitting
    if "-" in name:
        names.append(name.split("-")[1])
    else:
        # Handle cases where the delimiter is not found
        names.append(name)  # or any other desired handling

print(names[:10]) # Printing some species


In [ ]:
# %% Visualizing some images of different Classes

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
# %% 4. Building a Model (Resnet50)

In [ ]:
# We are calling Resnet50 and adding some layers of our own
# Below is the Architecture of Resnet50

In [ ]:
Model_URL ='https://kaggle.com/models/google/resnet-v2/frameworks/TensorFlow2/variations/50-classification/versions/2'
model = Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(img_size, img_size, 3)),
    hub.KerasLayer(Model_URL),
    tf.keras.layers.Dense(10, activation="softmax")])
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)


In [ ]:
# Compiling the model in which we are using Categorical Crossentropy and the optimizer is Adam.

In [ ]:
model.compile(
      loss=tf.keras.losses.CategoricalCrossentropy(),
      optimizer=tf.keras.optimizers.Adam(),
      metrics=["accuracy"]
  )

In [ ]:
model.summary()

In [ ]:
# We will add 3 Very Important Callbacks for our model
# ----- ModelCheckpoint: This callback saves the model weights after every epoch if the validation loss improves.
# ----- EarlyStopping: This callback stops the training process if the validation loss doesn't improve for a certain number of epochs.
# ----- ReduceLROnPlateu: This callback monitors a quantity and if no improvement is seen for a 'patience' number of epochs, the learning rate is reduced.
# All of them prevent Overfitting and save the best model according to the validation loss.

In [ ]:
model_name = "model_reduit.keras"
checkpoint = tf.keras.callbacks.ModelCheckpoint(model_name,
                            monitor="val_loss",
                            mode="min",
                            save_best_only = True,
                            verbose=1)



earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta = 0, patience = 3,
                                                 verbose = 1, restore_best_weights=True)

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

In [ ]:
import tensorflow_hub as hub

history = model.fit(train, epochs=30, validation_data=val, callbacks=[checkpoint,earlystopping, reduce_lr])